<a href="https://colab.research.google.com/github/oleatree/test/blob/master/prozhito_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем все необходимые библиотеки.

In [ ]:
from google.colab import files
from IPython.display import clear_output

In [ ]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()

In [ ]:
%load_ext pycodestyle_magic

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Загружаем данные.

In [ ]:
whole_table_with_lemm = drive.CreateFile(
    {'id': '1rKrNCmgWPReJujCoZCDlhAb66yiUZ_by'})
whole_table_with_lemm.GetContentFile('whole_table_with_lemm.csv')

Импортируем все необходимые библиотеки.

In [ ]:
import re
import nltk
import joblib

import pandas as pd
import numpy as np

from pprint import pprint
from ast import literal_eval
from datetime import datetime
from string import punctuation

from tqdm.notebook import tqdm

from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

clear_output()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('russian'))

# Цель исследования
Тематическое моделирование корпуса текстов: связь тем дневниковых записей с периодом времени написания.
# Данные
**Meet your dataset: дневниковые записи** [(источник)](https://t.me/c/1358367494/782)<br>

**Откуда у нас дневники?**<br>
Наши данные - это причёсанный дамп [сайта "Прожито"](https://prozhito.org) от апреля 2019 года. Таблицы содержит несколько сотен тысяч записей за большой отрезок времени (от XVIII до XXI века, преимущественно — XX век), так что вам будет, где развернуться ;).<br>
<br>
**Как устроен датасет?**<br>
У нас есть две таблицы, `whole_table.csv` и `whole_table_with_lemm.csv`. Вторая отличается наличием колонки c лемматизированными (т.е. с приведенными в начальную форму словами) mystem записями.<br>
В обеих таблицах есть колонка:
*   `notes` - содержит сами дневниковые записи
*   `dates` - дата записи в формате год/месяц/день
*   `id`  -  айдишник автора (не записи!)
*   `author` - имя автора записи
<br>

Практически все поля заполнены, у некоторых отсутствует дата (так как таблица отсортирована по датам, они в самом начале).<br>
*От авторов исследования: даты отформатированы как YYYY/(M)M/(D)D, отсутствовать могут день, месяц или год, отсутствующие элементы даты заменены нулями.*
# Препроцессинг

In [ ]:
# напишем отдельную функцию парсинга дат для записей раньше 20 века
# или записей у которых отсутствует год, месяц или число
# исходя из целей исследования, такие записи неинформативны и их нужно удалять

def date_parser(date):
    if date and re.match(r'^19\d{2}/[1-9]\d?/[1-9]\d?$', date):
        date_split = [int(elem) for elem in date.split('/')]
        if date_split[0] >= 1905:
            return datetime(year=date_split[0],
                            month=date_split[1],
                            day=date_split[2])
    return np.nan

In [ ]:
df = pd.read_csv('whole_table_with_lemm.csv', sep='\t', index_col=0,
                 usecols=[0, 2, 3], parse_dates=['dates'],
                 date_parser=date_parser).rename(
                     columns={'notes\n': 'lemm', 'dates': 'date'})

In [ ]:
df.head()

,lemm,date
0,воскресение. \nвчера приезжать слава навеща...,NaT
1,"ночь мы достигать банка антон-дорн, в датский ...",NaT
2,суббота. москва. ехать в селище.\n,NaT
3,18 август его королевский величество отправлят...,NaT
4,"на следующий день, 19 август, прибывать в остр...",NaT


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384526 entries, 0 to 384525
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   lemm    384525 non-null  object        
 1   date    302547 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 8.8+ MB


In [ ]:
!rm whole_table_with_lemm.csv

Удаляем ненужные записи, делаем столбец `date` индексом.

In [ ]:
df = df.dropna().set_index('date')

In [ ]:
df.head()

,lemm
date,
1905-01-01,"в 7 час утро — 4°, туман с дым, тихо. сегодня ..."
1905-01-01,с ранний утро японец опять открывать огонь раз...
1905-01-01,день новый год проходить благополучно. японец ...
1905-01-01,"вчера говорить, что поражать перемена, который..."
1905-01-01,воскресение. японский выя год \nобычный вос...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 302546 entries, 1905-01-01 to 1999-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lemm    302546 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


## Токенизация

In [ ]:
punctuation + '— \n'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~— \n'

In [ ]:
regex = r'[!\"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~— \n]+'

In [ ]:
def tokenization(text):
    return [elem for elem in re.split(regex, text)
            if elem and elem not in stop_words]

In [ ]:
df['lemm'] = df['lemm'].apply(tokenization)
clear_output()

In [ ]:
df.head()

,lemm
date,
1905-01-01,"[7, час, утро, 4°, туман, дым, тихо, сегодня, ..."
1905-01-01,"[ранний, утро, японец, открывать, огонь, разом..."
1905-01-01,"[день, новый, год, проходить, благополучно, яп..."
1905-01-01,"[вчера, говорить, поражать, перемена, который,..."
1905-01-01,"[воскресение, японский, выя, год, обычный, вос..."


Сохраняем и загружаем на гуглдиск готовые к topic modelling данные.

In [ ]:
df.to_csv('prozhito_lemmas.csv')

In [ ]:
!zip prozhito_lemmas.zip prozhito_lemmas.csv

  adding: prozhito_lemmas.csv (deflated 78%)


In [ ]:
!rm prozhito_lemmas.csv

In [ ]:
files.download('prozhito_lemmas.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки готовой к topic modelling таблицы

# prozhito_lemmas = drive.CreateFile(
#    {'id': '1_t6vdR2iREGMdNNtuYMc09anV0YpbaSs'})
# prozhito_lemmas.GetContentFile('prozhito_lemmas.zip')

Функция для загруженного из гуглдиска датафрейма.

In [ ]:
def data_to_list(filename):
    df = pd.read_csv(filename)
    df['lemm'] = df['lemm'].apply(literal_eval)
    clear_output()
    return df

## Периодизация
Поскольку основная часть текстов корпуса приходится на советскую эпоху, решено было выделить следующие периоды:
*   *янв. 1905 – янв. 1907* (первая революция)
*   *февр. 1907 – май 1914* (период между революциями)
*   *июнь 1914 – октябрь 1918* (Первая мировая война и две главные революции в России)
*   *ноябрь 1918 – декабрь 1927* (гражданская война, продразверстка, НЭП)
*   *январь 1928 – август 1939* (коллективизация, Большой террор)
*   *сентябрь 1939 – май 1945* (Вторая мировая война)
*   *1946 – 1963* (промежуточный период истории, заканчивается «оттепелью»)
*   *1964 – май 1987* (застой)
*   *июнь 1987 – июнь 1990* (перестройка и развал СССР)
*   *1991 – 2001* (после СССР, «лихие 90-е»)

In [ ]:
df = [
    df.loc[
        pd.Timestamp('1905-1-1 00:00:00'): pd.Timestamp('1907-1-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1907-2-1 00:00:00'): pd.Timestamp('1914-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1914-6-1 00:00:00'): pd.Timestamp('1918-10-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1918-11-1 00:00:00'): pd.Timestamp('1927-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1928-1-1 00:00:00'): pd.Timestamp('1939-8-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1939-9-1 00:00:00'): pd.Timestamp('1945-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1946-1-1 00:00:00'): pd.Timestamp('1963-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1964-1-1 00:00:00'): pd.Timestamp('1987-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1987-6-1 00:00:00'): pd.Timestamp('1990-6-30 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1991-1-1 00:00:00'): pd.Timestamp('2001-12-31 23:59:59')]
       ]

In [ ]:
df[1].head()

,date,lemm
0,1907-02-01,"[пятница, чудный, погода, 1, солнце, ветер, па..."
1,1907-02-01,"[вчера, ната, звонить, шлиппе, сказать, сергей..."
2,1907-02-01,"[пятница, приходиться, позвать, шенгелидзе, т,..."
3,1907-02-01,"[четверг, писать, 2, февраль, доклад, военный,..."
4,1907-02-01,"[тетя, васино, рождение, тетя, мила, ходить, к..."


In [ ]:
df[1]['lemm'][0]

['пятница',
 'чудный',
 'погода',
 '1',
 'солнце',
 'ветер',
 'пахнуть',
 'весна',
 'это',
 'удивительно',
 'приятно',
 'завтрак',
 'лена',
 'ходить',
 'гулять',
 'обед',
 'время',
 'проходить',
 'совершенно',
 'бесцельно',
 'лень',
 'приниматься',
 'бродить',
 'угол',
 'угол',
 'таланцев',
 'пробирать',
 'немножко',
 'митя',
 'лень',
 'решать',
 'вообще',
 'обращаться',
 'строгий',
 'позволять',
 'александра',
 'арсеньевна',
 'рассказывать',
 'нижний',
 'думать',
 'открывать',
 'женский',
 'педагогический',
 'курсы',
 'хороший',
 'непременно',
 'поступать',
 'хотеться',
 'учиться',
 'вечер',
 'лихорадка',
 '37',
 '5',
 'таки',
 'пойти',
 'театр',
 'лена',
 'вася',
 'концерт',
 'польза',
 'общежитие',
 'гимназия',
 'концерт',
 'производить',
 'ровно',
 'никакой',
 'впечатление',
 'скучно',
 'антракт',
 'видеть',
 'эрю',
 'зина',
 'марья',
 'миха',
 'многий',
 'знакомый']

In [ ]:
period = ['first revolution', 'between revolutions', 'WWI', 'civil war',
          'great purge', 'WWII', 'before ottepel', 'stagnation','perestroyka',
          '90s']

In [ ]:
for i, time in enumerate(period):
    print(f'{time}:{" " * (20 - len(time))}\t{df[i].shape[0]} entries')

first revolution:    	5821 entries
between revolutions: 	18761 entries
WWI:                 	20362 entries
civil war:           	35981 entries
great purge:         	37175 entries
WWII:                	62686 entries
before ottepel:      	39742 entries
stagnation:          	58121 entries
perestroyka:         	6723 entries
90s:                 	13283 entries


Сохраняем и загружаем на гуглдиск разбитые на периоды данные.

In [ ]:
for i, table in tqdm(enumerate(df)):
    table.to_csv(f'{i + 1}_{period[i].replace(" ", "_")}.csv')

In [ ]:
!zip prozhito_periodization.zip *.csv

  adding: 10_90s.csv (deflated 78%)
  adding: 1_first_revolution.csv (deflated 78%)
  adding: 2_between_revolutions.csv (deflated 78%)
  adding: 3_WWI.csv (deflated 78%)
  adding: 4_civil_war.csv (deflated 77%)
  adding: 5_great_purge.csv (deflated 78%)
  adding: 6_WWII.csv (deflated 78%)
  adding: 7_before_ottepel.csv (deflated 78%)
  adding: 8_stagnation.csv (deflated 78%)
  adding: 9_perestroyka.csv (deflated 78%)


In [ ]:
files.download('prozhito_periodization.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки архива таблиц для каждого периода

# prozhito_periodization = drive.CreateFile(
#    {'id': '1LcrY9pNxXHa9xLJ4jGnm8lXVbegFodUQ'})
# prozhito_periodization.GetContentFile('prozhito_periodization.zip')

# LDA with Gensim

In [ ]:
models = []

for table in df:
    data = list(table['lemm'])
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    model = LdaModel(corpus, num_topics=10, id2word=id2word, passes=15,
                     iterations=100, per_word_topics=True)
    models.append(model)
    clear_output()
    print(f'Моделей обучено: {len(models)}')

Сохраняем и загружаем на гуглдиск обученные модели с наборами тем для каждого периода.

In [ ]:
joblib.dump(models, 'prozhito_topic_modelling.pkl')

In [ ]:
!zip prozhito_topic_modelling.zip prozhito_topic_modelling.pkl

In [ ]:
files.download('prozhito_periodization.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки обученных моделей

# prozhito_topic_modelling = drive.CreateFile(
#    {'id': '1rKrNCmgWPReJujCoZCDlhAb66yiUZ_by'})
# prozhito_topic_modelling.GetContentFile('prozhito_topic_modelling.pkl')

# Результаты
[Визуализация результатов](https://colab.research.google.com/drive/1rejmaQAdDrRG6KLD1cb4T382OfaVs_Rh?usp=sharing)
## Первая революция
### (*янв. 1905 – янв. 1907*)
Темы дневниковых записей:

In [ ]:
pprint(models[0].print_topics())

## Период между революциями
### (*февр. 1907 – май 1914*)
Темы дневниковых записей:

In [ ]:
pprint(models[1].print_topics())

## Первая мировая война и две главные революции в России
###(*июнь 1914 – октябрь 1918*)
Темы дневниковых записей:

In [ ]:
pprint(models[2].print_topics())

## Гражданская война, продразверстка, НЭП
### (*ноябрь 1918 – декабрь 1927*)
Темы дневниковых записей:

In [ ]:
pprint(models[3].print_topics())

## Коллективизация, Большой террор
### (*январь 1928 – август 1939*)
Темы дневниковых записей:

In [ ]:
pprint(models[4].print_topics())

## Вторая мировая война
### (*сентябрь 1939 – май 1945*)
Темы дневниковых записей:

In [ ]:
pprint(models[5].print_topics())

## Промежуточный период истории, заканчивается «оттепелью» 
### (*1946 – 1963*)
Темы дневниковых записей:

In [ ]:
pprint(models[6].print_topics())

## Застой
### (*1964 – май 1987*)
Темы дневниковых записей:

In [ ]:
pprint(models[7].print_topics())

## Перестройка и развал СССР
### (*июнь 1987 – июнь 1990*)
Темы дневниковых записей:

In [ ]:
pprint(models[8].print_topics())

## После СССР, «лихие 90-е»
### (*1991 – 2001*)
Темы дневниковых записей:

In [ ]:
pprint(models[9].print_topics())